<center>
    <h2>Online learning platform database - Neo4j</h2>
    <h3>Performing the queries and storing the queries execution time</h3>
</center>

<h3>Python - Neo4j interaction</h3>

Prior to performing the queries, I import the required modules (the <i>Neo4j</i> Python driver and the <i>time</i> and <i>csv</i> modules) and set four driver objects to which I pass the parameters to establish a connection to each of the four Neo4j instances running in Docker.

In [1]:
# import modules
from neo4j import GraphDatabase     # Neo4j driver
import time                         # time-related functions to register query execution times
import csv                          # read and write csv files

# create driver object
driver = GraphDatabase.driver(uri = 'neo4j://127.0.0.1:7687', auth = ('neo4j', 'myPassword'))
driver2 = GraphDatabase.driver(uri = 'neo4j://127.0.0.1:7692', auth = ('neo4j', 'myPassword'))
driver3 = GraphDatabase.driver(uri = 'neo4j://127.0.0.1:7693', auth = ('neo4j', 'myPassword'))
driver4 = GraphDatabase.driver(uri = 'neo4j://127.0.0.1:7694', auth = ('neo4j', 'myPassword'))

<h3>Query the Neo4j instances</h3>

I create a dictionary of lists for each of the four Neo4j instances. In these dictionaries the keys are the query names and the values are the 31 query execution times: in fact I attach the value of the query execution time of the most recent query to the list. Since query execution times are required in milliseconds, prior to attaching them, I multiply them by 1000 and round them to the fifth decimal precision.
As previously explained, the above summarized actions (for each of the four queries on each of the four instances) are performed by following a standard succession of steps. Each step is encapsulated within a notebook cell (so each query is performed 31 times by using three notebook cells), as follows:
 - step 1: define the query as a string and pass it together with the <code>database</code> parameter to the <code>execute_query()</code> method of the driver object, while contextually registering the time before and after the query execution. Finally print query results;
 - step 2: compute execution time of the first query execution and store it within the corresponding dictionary list;
 - step 3: [thirty times] perform query execution while creating prior and following timestamps, compute execution time and store it within the corresponding dictionary list.

For each Neo4j instance, after having performed the four queries, I will finally compute the mean of the query executions from step 3. Together with the first query execution, this mean value will be stored into a new dictionary, specific to a dataset. Originally, I would use these four new dictionaries to save the query execution times into a csv file for constructing histograms. I later resolved to save all the 31 recorded query execution times and pass them all to Microsoft© Excel to process them.

N.B.:
Contrary to what has been introduced when presenting the methodologies used for accessing and querying Neo4j, since I don't need to use the information contained in the <i>summary</i> object to get the query execution time, I am only interested in the query result records for showing them. I can use dot notation on the query object to access the query records (a query object has a <code>records</code> attribute). I could use the index <code>0</code> on the query object since the records are the first element in a list. By iterating on the records object and applying the <code>data()</code> method during iteration I can turn the records elements into key-value pairs (a dictionary, actually).

In [2]:
smallDict = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}
mediumDict = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}
largeDict = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}
humongousDict = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}

In [3]:
# mean function
def mean(aList):
    n = len(aList)
    sum = 0
    for value in aList:
        sum += value
    return sum / n

<h3>Neo4j instance with 250k hashes</h3>

I start with the smallest Neo4j instance.

<h4>Query 1</h4>

In [5]:
# step 1
small_neo4j1 = 'MATCH (s:student) -[:is_enrolled]-> (c:course) WHERE c.courseID = \'192\' RETURN s.firstName, s.lastName'

before = time.time()
small_query1 = driver.execute_query(small_neo4j1, database = 'neo4j')
after = time.time()

records = small_query1.records
for record in records:
    print(record.data()['s.firstName'], record.data()['s.lastName'],)

Custodia Hidalgo
Ledün Soylu
Nath Nicolas
Ana Narušis
Émile Nicolas
Sarah Lara
Patrícia Leite
Cathrine Lie
Arthur Laroche
Casandra Arenas
Narciso Ferrán
Vigilija Gaižauskas
Ingeborg Amundsen


In [6]:
# step 2
msec_duration = (after - before) * 1000
smallDict['query1'].append(round(msec_duration, 5))

In [7]:
# step 3
for i in range(0, 30):
    before = time.time()
    driver.execute_query(small_neo4j1, database = 'neo4j')
    after = time.time()
    msec_duration = (after - before) * 1000
    smallDict['query1'].append(round(msec_duration, 5))

<h4>Query 2</h4>

In [9]:
# step 1
small_neo4j2 = 'MATCH(c:course) WHERE c.discipline = \'statistics\' AND c.courseYear = \'2022\' RETURN c.courseName'

before = time.time()
small_query2 = driver.execute_query(small_neo4j2, database = 'neo4j')
after = time.time()

records = small_query2.records
for record in records:
    print(record.data()['c.courseName'])

Econometrics: Methods and Applications
Exploratory Data Analysis
Understanding Clinical Research: Behind the Statistics
Introduction to Probability and Data with R
Bayesian Statistics: From Concept to Data Analysis
Introduction to Statistics
Python and Statistics for Financial Analysis
Basic Statistics
Foundations: Data, Data, Everywhere


In [10]:
# step 2
msec_duration = (after - before) * 1000
smallDict['query2'].append(round(msec_duration, 5))

In [11]:
# step 3
for i in range(0, 30):
    before = time.time()
    driver.execute_query(small_neo4j2, database = 'neo4j')
    after = time.time()
    msec_duration = (after - before) * 1000
    smallDict['query2'].append(round(msec_duration, 5))

<h4>Query 3</h4>

In [13]:
# step 1
small_neo4j3 = 'MATCH (c:course {discipline: \'maths\'}) <-[:is_enrolled]- (s:student) -[:studies]-> (m:material {mType: \'lecture slides\'}) <-[:uses]- (c:course) WHERE right(s.email, 9) = \'gmail.com\' RETURN COUNT(m);'

before = time.time()
small_query3 = driver.execute_query(small_neo4j3, database = 'neo4j')
after = time.time()

records = small_query3.records
for record in records:
    print(record.data()['COUNT(m)'])

838


In [14]:
# step 2
msec_duration = (after - before) * 1000
smallDict['query3'].append(round(msec_duration, 5))

In [15]:
# step 3
for i in range(0, 30):
    before = time.time()
    driver.execute_query(small_neo4j3, database = 'neo4j')
    after = time.time()
    msec_duration = (after - before) * 1000
    smallDict['query3'].append(round(msec_duration, 5))

<h4>Query 4</h4>

In [17]:
# step 1
small_neo4j4 = 'MATCH (s:student) -[:is_enrolled]-> (c:course {discipline: \'psychology\'}) WHERE right(s.country, 4) = \'orea\' AND left(s.dob, 1) <> \'2\' RETURN DISTINCT s.firstName, s.lastName, s.country ORDER BY s.lastName;'

before = time.time()
small_query4 = driver.execute_query(small_neo4j4, database = 'neo4j')
after = time.time()

records = small_query4.records
for record in records:
    print(record.data()['s.firstName'], record.data()['s.lastName'], record.data()['s.country'])

Cathrine Lie South Korea
Lynda Reynolds Korea
Raghav Sura North Korea


In [18]:
# step 2
msec_duration = (after - before) * 1000
smallDict['query4'].append(round(msec_duration, 5))

In [19]:
# step 3
for i in range(0, 30):
    before = time.time()
    driver.execute_query(small_neo4j4, database = 'neo4j')
    after = time.time()
    msec_duration = (after - before) * 1000
    smallDict['query4'].append(round(msec_duration, 5))

In [21]:
smallDataset = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}
for key in smallDict:
    smallDataset[key].append(smallDict[key][0])
    mean30 = mean(smallDict[key][1 : 31])
    smallDataset[key].append(round(mean30, 5))
smallDataset

{'query1': [5489.99786, 24.541],
 'query2': [418.41102, 17.52477],
 'query3': [1242.91325, 39.00357],
 'query4': [449.72205, 14.5945]}

<h3>Neo4j instance with 500k hashes</h3>

<h4>Query 1</h4>

In [22]:
# step 1
medium_neo4j1 = 'MATCH (s:student) -[:is_enrolled]-> (c:course) WHERE c.courseID = \'192\' RETURN s.firstName, s.lastName'

before = time.time()
medium_query1 = driver2.execute_query(medium_neo4j1, database = 'neo4j')
after = time.time()

records = medium_query1.records
for record in records:
    print(record.data()['s.firstName'], record.data()['s.lastName'],)

Custodia Hidalgo
Ledün Soylu
Nath Nicolas
Sarah Lara
Karl Christensen
Christl Henschel
Casandra Arenas
Narciso Ferrán
Vigilija Gaižauskas
Nedas Naujokas
Ana Narušis
Émile Nicolas
Joris Kavaliauskas
Yuvaan Dara
Patrícia Leite
Cathrine Lie
Arthur Laroche
Ingeborg Amundsen
Débora Vaz
Miguel Real


In [23]:
# step 2
msec_duration = (after - before) * 1000
mediumDict['query1'].append(round(msec_duration, 5))

In [24]:
# step 3
for i in range(0, 30):
    before = time.time()
    driver2.execute_query(medium_neo4j1, database = 'neo4j')
    after = time.time()
    msec_duration = (after - before) * 1000
    mediumDict['query1'].append(round(msec_duration, 5))

<h4>Query 2</h4>

In [26]:
# step 1
medium_neo4j2 = 'MATCH(c:course) WHERE c.discipline = \'statistics\' AND c.courseYear = \'2022\' RETURN c.courseName'

before = time.time()
medium_query2 = driver2.execute_query(medium_neo4j2, database = 'neo4j')
after = time.time()

records = medium_query2.records
for record in records:
    print(record.data()['c.courseName'])

Econometrics: Methods and Applications
Exploratory Data Analysis
Understanding Clinical Research: Behind the Statistics
Introduction to Probability and Data with R
Bayesian Statistics: From Concept to Data Analysis
Introduction to Statistics
Python and Statistics for Financial Analysis
Basic Statistics
Foundations: Data, Data, Everywhere


In [27]:
# step 2
msec_duration = (after - before) * 1000
mediumDict['query2'].append(round(msec_duration, 5))

In [28]:
# step 3
for i in range(0, 30):
    before = time.time()
    driver2.execute_query(medium_neo4j2, database = 'neo4j')
    after = time.time()
    msec_duration = (after - before) * 1000
    mediumDict['query2'].append(round(msec_duration, 5))

<h4>Query 3</h4>

In [30]:
# step 1
medium_neo4j3 = 'MATCH (c:course {discipline: \'maths\'}) <-[:is_enrolled]- (s:student) -[:studies]-> (m:material {mType: \'lecture slides\'}) <-[:uses]- (c:course) WHERE right(s.email, 9) = \'gmail.com\' RETURN COUNT(m);'

before = time.time()
medium_query3 = driver2.execute_query(medium_neo4j3, database = 'neo4j')
after = time.time()

records = medium_query3.records
for record in records:
    print(record.data()['COUNT(m)'])

1698


In [31]:
# step 2
msec_duration = (after - before) * 1000
mediumDict['query3'].append(round(msec_duration, 5))

In [32]:
# step 3
for i in range(0, 30):
    before = time.time()
    driver2.execute_query(medium_neo4j3, database = 'neo4j')
    after = time.time()
    msec_duration = (after - before) * 1000
    mediumDict['query3'].append(round(msec_duration, 5))

<h4>Query 4</h4>

In [34]:
# step 1
medium_neo4j4 = 'MATCH (s:student) -[:is_enrolled]-> (c:course {discipline: \'psychology\'}) WHERE right(s.country, 4) = \'orea\' AND left(s.dob, 1) <> \'2\' RETURN DISTINCT s.firstName, s.lastName, s.country ORDER BY s.lastName;'

before = time.time()
medium_query4 = driver2.execute_query(medium_neo4j4, database = 'neo4j')
after = time.time()

records = medium_query4.records
for record in records:
    print(record.data()['s.firstName'], record.data()['s.lastName'], record.data()['s.country'])

Ninthe Horrocks Noord-Korea
Cathrine Lie South Korea
Miguel Real República de Corea
Lynda Reynolds Korea
Raghav Sura North Korea


In [35]:
# step 2
msec_duration = (after - before) * 1000
mediumDict['query4'].append(round(msec_duration, 5))

In [36]:
# step 3
for i in range(0, 30):
    before = time.time()
    driver2.execute_query(medium_neo4j4, database = 'neo4j')
    after = time.time()
    msec_duration = (after - before) * 1000
    mediumDict['query4'].append(round(msec_duration, 5))

In [38]:
mediumDataset = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}
for key in mediumDict:
    mediumDataset[key].append(mediumDict[key][0])
    mean30 = mean(mediumDict[key][1 : 31])
    mediumDataset[key].append(round(mean30, 5))
mediumDataset

{'query1': [5931.72693, 24.64457],
 'query2': [276.00718, 20.78204],
 'query3': [1381.90222, 39.59148],
 'query4': [504.84109, 19.65367]}

<h3>Neo4j instance with 750k hashes</h3>

<h4>Query 1</h4>

In [39]:
# step 1
large_neo4j1 = 'MATCH (s:student) -[:is_enrolled]-> (c:course) WHERE c.courseID = \'192\' RETURN s.firstName, s.lastName'

before = time.time()
large_query1 = driver3.execute_query(large_neo4j1, database = 'neo4j')
after = time.time()

records = large_query1.records
for record in records:
    print(record.data()['s.firstName'], record.data()['s.lastName'],)

Custodia Hidalgo
Sarah Lara
Narciso Ferrán
Patrícia Leite
Vigilija Gaižauskas
Casandra Arenas
Ledün Soylu
Arthur Laroche
Ana Narušis
Nath Nicolas
Émile Nicolas
Cathrine Lie
Ingeborg Amundsen
Nedas Naujokas
Christl Henschel
Miguel Real
Karl Christensen
Joris Kavaliauskas
Yuvaan Dara
Débora Vaz
Urvi Dani
Collin Heerkens
Brian Thompson
Özkutlu Gül
Dorita Abella
Liliana Flaiano


In [40]:
# step 2
msec_duration = (after - before) * 1000
largeDict['query1'].append(round(msec_duration, 5))

In [41]:
# step 3
for i in range(0, 30):
    before = time.time()
    driver3.execute_query(large_neo4j1, database = 'neo4j')
    after = time.time()
    msec_duration = (after - before) * 1000
    largeDict['query1'].append(round(msec_duration, 5))

<h4>Query 2</h4>

In [43]:
# step 1
large_neo4j2 = 'MATCH(c:course) WHERE c.discipline = \'statistics\' AND c.courseYear = \'2022\' RETURN c.courseName'

before = time.time()
large_query2 = driver3.execute_query(large_neo4j2, database = 'neo4j')
after = time.time()

records = large_query2.records
for record in records:
    print(record.data()['c.courseName'])

Econometrics: Methods and Applications
Exploratory Data Analysis
Understanding Clinical Research: Behind the Statistics
Introduction to Probability and Data with R
Bayesian Statistics: From Concept to Data Analysis
Introduction to Statistics
Python and Statistics for Financial Analysis
Basic Statistics
Foundations: Data, Data, Everywhere


In [44]:
# step 2
msec_duration = (after - before) * 1000
largeDict['query2'].append(round(msec_duration, 5))

In [45]:
# step 3
for i in range(0, 30):
    before = time.time()
    driver3.execute_query(large_neo4j2, database = 'neo4j')
    after = time.time()
    msec_duration = (after - before) * 1000
    largeDict['query2'].append(round(msec_duration, 5))

<h4>Query 3</h4>

In [47]:
# step 1
large_neo4j3 = 'MATCH (c:course {discipline: \'maths\'}) <-[:is_enrolled]- (s:student) -[:studies]-> (m:material {mType: \'lecture slides\'}) <-[:uses]- (c:course) WHERE right(s.email, 9) = \'gmail.com\' RETURN COUNT(m);'

before = time.time()
large_query3 = driver3.execute_query(large_neo4j3, database = 'neo4j')
after = time.time()

records = large_query3.records
for record in records:
    print(record.data()['COUNT(m)'])

2628


In [48]:
# step 2
msec_duration = (after - before) * 1000
largeDict['query3'].append(round(msec_duration, 5))

In [49]:
# step 3
for i in range(0, 30):
    before = time.time()
    driver3.execute_query(large_neo4j3, database = 'neo4j')
    after = time.time()
    msec_duration = (after - before) * 1000
    largeDict['query3'].append(round(msec_duration, 5))

<h4>Query 4</h4>

In [51]:
# step 1
large_neo4j4 = 'MATCH (s:student) -[:is_enrolled]-> (c:course {discipline: \'psychology\'}) WHERE right(s.country, 4) = \'orea\' AND left(s.dob, 1) <> \'2\' RETURN DISTINCT s.firstName, s.lastName, s.country ORDER BY s.lastName;'

before = time.time()
large_query4 = driver3.execute_query(large_neo4j4, database = 'neo4j')
after = time.time()

records = large_query4.records
for record in records:
    print(record.data()['s.firstName'], record.data()['s.lastName'], record.data()['s.country'])

Tere Castells República Popular Democrática de Corea
Ninthe Horrocks Noord-Korea
Cathrine Lie South Korea
Miguel Real República de Corea
Lynda Reynolds Korea
Raghav Sura North Korea


In [52]:
# step 2
msec_duration = (after - before) * 1000
largeDict['query4'].append(round(msec_duration, 5))

In [53]:
# step 3
for i in range(0, 30):
    before = time.time()
    driver3.execute_query(large_neo4j4, database = 'neo4j')
    after = time.time()
    msec_duration = (after - before) * 1000
    largeDict['query4'].append(round(msec_duration, 5))

In [55]:
largeDataset = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}
for key in largeDict:
    largeDataset[key].append(largeDict[key][0])
    mean30 = mean(largeDict[key][1 : 31])
    largeDataset[key].append(round(mean30, 5))
largeDataset

{'query1': [4215.55996, 23.35128],
 'query2': [314.49819, 15.97649],
 'query3': [1225.1122, 45.31332],
 'query4': [434.03172, 15.21886]}

<h3>Neo4j instance with 1m hashes</h3>

<h4>Query 1</h4>

In [56]:
# step 1
humongous_neo4j1 = 'MATCH (s:student) -[:is_enrolled]-> (c:course) WHERE c.courseID = \'192\' RETURN s.firstName, s.lastName'

before = time.time()
humongous_query1 = driver4.execute_query(humongous_neo4j1, database = 'neo4j')
after = time.time()

records = humongous_query1.records
for record in records:
    print(record.data()['s.firstName'], record.data()['s.lastName'],)

Custodia Hidalgo
Sarah Lara
Narciso Ferrán
Patrícia Leite
Vigilija Gaižauskas
Casandra Arenas
Ledün Soylu
Arthur Laroche
Ana Narušis
Nath Nicolas
Émile Nicolas
Cathrine Lie
Ingeborg Amundsen
Nedas Naujokas
Christl Henschel
Miguel Real
Karl Christensen
Joris Kavaliauskas
Yuvaan Dara
Débora Vaz
Urvi Dani
Collin Heerkens
Özkutlu Gül
Brian Thompson
Dorita Abella
Liliana Flaiano
Finn Karlsen
David Miranda
Torsten Schulz
Kristen Webb
Shaan Raju
Giuseppina Scarfoglio
Mamen Teruel
Eduardo Rezende
Melania Savorgnan


In [57]:
# step 2
msec_duration = (after - before) * 1000
humongousDict['query1'].append(round(msec_duration, 5))

In [58]:
# step 3
for i in range(0, 30):
    before = time.time()
    driver4.execute_query(humongous_neo4j1, database = 'neo4j')
    after = time.time()
    msec_duration = (after - before) * 1000
    humongousDict['query1'].append(round(msec_duration, 5))

<h4>Query 2</h4>

In [60]:
# step 1
humongous_neo4j2 = 'MATCH(c:course) WHERE c.discipline = \'statistics\' AND c.courseYear = \'2022\' RETURN c.courseName'

before = time.time()
humongous_query2 = driver4.execute_query(humongous_neo4j2, database = 'neo4j')
after = time.time()

records = humongous_query2.records
for record in records:
    print(record.data()['c.courseName'])

Econometrics: Methods and Applications
Exploratory Data Analysis
Understanding Clinical Research: Behind the Statistics
Introduction to Probability and Data with R
Bayesian Statistics: From Concept to Data Analysis
Introduction to Statistics
Python and Statistics for Financial Analysis
Basic Statistics
Foundations: Data, Data, Everywhere


In [61]:
# step 2
msec_duration = (after - before) * 1000
humongousDict['query2'].append(round(msec_duration, 5))

In [62]:
# step 3
for i in range(0, 30):
    before = time.time()
    driver4.execute_query(humongous_neo4j2, database = 'neo4j')
    after = time.time()
    msec_duration = (after - before) * 1000
    humongousDict['query2'].append(round(msec_duration, 5))

<h4>Query 3</h4>

In [64]:
# step 1
humongous_neo4j3 = 'MATCH (c:course {discipline: \'maths\'}) <-[:is_enrolled]- (s:student) -[:studies]-> (m:material {mType: \'lecture slides\'}) <-[:uses]- (c:course) WHERE right(s.email, 9) = \'gmail.com\' RETURN COUNT(m);'

before = time.time()
humongous_query3 = driver4.execute_query(humongous_neo4j3, database = 'neo4j')
after = time.time()

records = humongous_query3.records
for record in records:
    print(record.data()['COUNT(m)'])

3498


In [65]:
# step 2
msec_duration = (after - before) * 1000
humongousDict['query3'].append(round(msec_duration, 5))

In [66]:
# step 3
for i in range(0, 30):
    before = time.time()
    driver4.execute_query(humongous_neo4j3, database = 'neo4j')
    after = time.time()
    msec_duration = (after - before) * 1000
    humongousDict['query3'].append(round(msec_duration, 5))

<h4>Query 4</h4>

In [68]:
# step 1
humongous_neo4j4 = 'MATCH (s:student) -[:is_enrolled]-> (c:course {discipline: \'psychology\'}) WHERE right(s.country, 4) = \'orea\' AND left(s.dob, 1) <> \'2\' RETURN DISTINCT s.firstName, s.lastName, s.country ORDER BY s.lastName;'

before = time.time()
humongous_query4 = driver4.execute_query(humongous_neo4j4, database = 'neo4j')
after = time.time()

records = humongous_query4.records
for record in records:
    print(record.data()['s.firstName'], record.data()['s.lastName'], record.data()['s.country'])

Tere Castells República Popular Democrática de Corea
Leila Gailys Korea
Ninthe Horrocks Noord-Korea
Cathrine Lie South Korea
Miguel Real República de Corea
Lynda Reynolds Korea
Debra Shaw Korea
Raghav Sura North Korea


In [69]:
# step 2
msec_duration = (after - before) * 1000
humongousDict['query4'].append(round(msec_duration, 5))

In [70]:
# step 3
for i in range(0, 30):
    before = time.time()
    driver4.execute_query(humongous_neo4j4, database = 'neo4j')
    after = time.time()
    msec_duration = (after - before) * 1000
    humongousDict['query4'].append(round(msec_duration, 5))

In [72]:
humongousDataset = {'query1' : list(), 'query2' : list(), 'query3' : list(), 'query4' : list()}
for key in humongousDict:
    humongousDataset[key].append(humongousDict[key][0])
    mean30 = mean(humongousDict[key][1 : 31])
    humongousDataset[key].append(round(mean30, 5))
humongousDataset

{'query1': [5255.56993, 23.6992],
 'query2': [338.23609, 16.86485],
 'query3': [1648.39602, 128.11236],
 'query4': [704.70405, 18.21342]}

In [73]:
with open('neo4j_tests.csv', 'w', newline = '') as neo4j_tests:
    writer = csv.writer(neo4j_tests, delimiter = ',')
    keys = smallDict.keys()
    limit = len(smallDict['query1'])
    
    writer.writerow(keys)
    writer.writerow('s') # s stands for small dataset
    for i in range(0, limit):
        writer.writerow(smallDict[k][i] for k in keys)
    writer.writerow('m')  # m stands for medium dataset
    for i in range(0, limit):
        writer.writerow(mediumDict[k][i] for k in keys)
    writer.writerow('l') # l stands for large dataset
    for i in range(0, limit):
        writer.writerow(largeDict[k][i] for k in keys)
    writer.writerow('h') # h stands for humongous dataset
    for i in range(0, limit):
        writer.writerow(humongousDict[k][i] for k in keys)